In [71]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

Make data: sample 10000 shots from each year.

In [37]:
values_counts = []
for year in range(2003,2017):
    data = pd.read_csv('data/%d.csv' % (year,))
    if year==2003:
        df = data.loc[0:2,:]
    df = df.append(data.iloc[np.random.choice(range(len(data)),size=10000,replace=False)])
df = df.drop(df.head(n=3).index,axis=0)

First specification - Categories: fairway (with teebox), rough (with fringe), bunker, green, other

In [45]:
df['From_Location(Scorer)'].value_counts(dropna=False)/len(df)

Green                 0.414021
Tee Box               0.255136
Fairway               0.169214
Primary Rough         0.079243
Green Side Bunker     0.024743
Intermediate Rough    0.023221
Fringe                0.017736
Fairway Bunker        0.011879
Native Area           0.003643
Other                 0.001043
Grass Bunker          0.000071
Water                 0.000050
Name: From_Location(Scorer), dtype: float64

In [64]:
df.insert(len(df.columns),'Cat',['Other']*len(df))

In [65]:
df.loc[df['From_Location(Scorer)'] == 'Green','Cat'] = 'Green'
df.loc[df['From_Location(Scorer)'].isin(['Tee Box','Fairway']),'Cat'] = 'Fairway'
df.loc[df['From_Location(Scorer)'].isin(['Primary Rough','Intermediate Rough','Fringe']),'Cat'] = 'Rough'
df.loc[df['From_Location(Scorer)'].isin(['Green Side Bunker','Fairway Bunker']),'Cat'] = 'Bunker'

In [70]:
df.Cat.value_counts(dropna=False)

Fairway    59409
Green      57963
Rough      16828
Bunker      5127
Other        673
Name: Cat, dtype: int64

In [ ]:
model = smf.ols('')